# Generación de datos sintéticos con *Information Preserving Statistical Obfuscation* (IPSO)

En este cuaderno se muestra un ejemplo de creación de datos sintéticos siguiendo el método de *Information Preserving Statistical Obfuscation* (IPSO) en el lenguaje R. Este método produce datos sintéticos que preserva varianzas, covarianzas y valores ajustados. 

Con este método, presupone que los datos originales están conformados por dos subconjuntos de variables: la matriz $X$ con variables no confidenciales y la matriz $Y$ con variables confidenciales. Se ajusta un modelo $Y = \beta X + \varepsilon $ el cual se usa para generar datos sintéticos $Y'$

## Instalación de biliotecas 

Para este ejercicio se hace uso de la biblioteca [RegSDC](https://cran.r-project.org/web/packages/RegSDC/index.html) la cual ofrece las funciones para aplicar IPSO. También se hace uso de la biblioteca minio.s3 para el acceso al lago de datos y las bibliotecas de tydiverse para manejar los datos. 


In [ ]:
library(devtools)
install_github("nagdevAmruthnath/minio.s3")
install.packages("RegSDC")
install.packages("tidyverse")
install.packages("synthpop")

## Importación de bibliotecas

In [ ]:
library(RegSDC)
library(minio.s3)
library(tidyverse)
library(tidymodels)

## Acceso a los datos del lago

In [ ]:
Sys.setenv("AWS_ACCESS_KEY_ID" = readline(prompt = "Usuario: "),
           "AWS_SECRET_ACCESS_KEY" = getPass(prompt = "Contraseña: "),
          "AWS_S3_ENDPOINT"= "lcidn4.inegi.gob.mx:9100"
          )
ruta_archivo <- "s3://hostname/nombredearchivo.ext"

## Variables de interés 

Para este ejercicio no requerimos hacer uso de FAC el cual indica el factor de expansión. Por lo que las variables de interés serán:

- **LOC:** Localidad
- **MUN:** Municipio
- **T_LOC:** Tamaño de localidad 
- **MAN:** Manzana
- **CD_A:** Ciudad auto representada
- **ENT:** Entidad
- **AGEB:** Área GeoEstadística Básica
- **SEX:** Sexo
- **EDA:** Edad 
- **NAC_DIA:** Dia de nacimiento
- **NAC_MES:** Mes de nacimiento
- **NAC_ANIO:** Año de nacimiento 
- **CS_P13_1:** Nivel escolar
- **POS_OCU:** Posición en la ocupación
- **INGOCUP:** Ingreso del personal ocupado 

Como se puede observar, las variables son en general variables de identificación o variables sensibles. Por lo que la aplicación de datos sintéticos resulta ideal.

In [ ]:
variables_interes <- c('LOC','MUN','T_LOC','MAN','CD_A','ENT','AGEB','SEX','EDA','NAC_DIA','NAC_MES','NAC_ANIO','CS_P13_1','POS_OCU','INGOCUP')
variables_interes

La tabla original se limita a las variables de interés, dentro del mismo proceso se elige el tipo de variables. 
La mayoría de los datos son tipo factor. Debido a que representan un numero finito de clases donde no tiene sentido analizar el valor numérico.
La excepción resulta las variables EDA, NAC_DIA, NAC_MES, NAC_ANIO, INGOCUP las cuales si hacen referencia a un valor numérico. 

También se filtra este ejercicio a registros donde el año de nacimiento sea distinto de 9999 (se desconoce el año de nacimiento) y el día de nacimiento distinto de 99 (se desconoce el día de nacimiento). Ya que al ser un método de regresión, estos valores contaminarán el modelo. Además, se omiten los registros que tienen valores faltantes.  

Adicionalmente se realiza un muestreo aleatorio con solo el 1% de los datos, esto permite obtener resultados más rápidos. Una vez establecido el modelo final, es necesario utilizar la tabla de datos completa.

In [ ]:
# Establecemos una semilla para que los resultados sean reproducibles
set.seed(2022)
sdem_interes  <- tabla_sdem %>% 
    select(all_of(variables_interes))  %>%
    mutate_at( 
        c(
            'LOC','MUN','T_LOC','MAN','CD_A','ENT','AGEB','SEX','CS_P13_1','POS_OCU'
        ),
        factor
    ) %>% 
    mutate_at( 
        c(
            'EDA','NAC_DIA','NAC_MES','NAC_ANIO','INGOCUP'
        ),
        as.numeric
    ) %>% 
    filter (
        ! NAC_ANIO == 9999,
        ! NAC_DIA == 99,
        ! is.na(CS_P13_1) 
    ) %>% 
    slice_sample(prop=0.01,replace=FALSE)

head(sdem_interes)

### Selección de variables confidenciales y no confidenciales

Para este ejercicio seleccionamos las variables confidenciales:

- **LOC:** Localidad
- **MUN:** Municipio
- **T_LOC:** Tamaño de localidad 
- **MAN:** Manzana
- **CD_A:** Ciudad auto representada
- **ENT:** Entidad
- **AGEB:** Área GeoEstadística Básica

las cuales formaran nuestra matriz $X$. Por lo tanto, las variables no confidenciales que forman la matriz $Y$ serán:

- **SEX:** Sexo
- **EDA:** Edad 
- **NAC_DIA:** Dia de nacimiento
- **NAC_MES:** Mes de nacimiento
- **NAC_ANIO:** Año de nacimiento 
- **CS_P13_1:** Nivel escolar
- **POS_OCU:** Posición en la ocupación
- **INGOCUP:** Ingreso del personal ocupado


In [ ]:
X <- sdem_interes %>% select (all_of(c('LOC','MUN','T_LOC','MAN','CD_A','ENT','AGEB')))
head(X) 

In [ ]:
Y <- sdem_interes %>% select (all_of(c('SEX','POS_OCU','EDA','NAC_DIA','NAC_MES','NAC_ANIO','CS_P13_1','INGOCUP')))
head(Y) 

### Preprocesamiento de datos

debido a que tenemos variables del tipo categóricas, es necesario crear columnas de tipo dummy las cuales indican con un 1 si el registro pertenece a este valor en la variable y 0 en el caso contrario. A este proceso se le denomina *one hot encoding*.



In [ ]:
X_onehot = model.matrix(~ LOC + MUN + T_LOC + MAN + CD_A+ ENT+ AGEB, data = X)
X_onehot

In [ ]:
Y_onehot = model.matrix(~ 0+SEX+POS_OCU+EDA+NAC_DIA+NAC_MES+NAC_ANIO+CS_P13_1+INGOCUP, data = Y)

# por defecto este sistema omite el primer valor de las variables categóricas
# Para Y es necesario también tener una columna para esta variable, por lo que se hace el siguiente proceso
old_contr <- options("contrasts")$contrasts
new_contr <- old_contr
new_contr["unordered"] <- "contr_one_hot"
options(contrasts = new_contr)

Y_onehot = model.matrix(~ 0+SEX+POS_OCU+EDA+NAC_DIA+NAC_MES+NAC_ANIO+CS_P13_1+INGOCUP, data = Y)
Y_onehot

## Creación del modelo 

Creamos el modelo, adicionalmente medimos el tiempo que tomó crearlo

In [ ]:
system.time( 
    modelo <- RegSDCipso(Y_onehot, X_onehot, ensureIntercept = TRUE) %>%
        as_tibble()
)

In [ ]:
modelo

### Postprocesamiento de datos

Como podemos ver en el modelo creado, los valores creados no se parecen nada a los datos originales, por lo que es necesario procesarlos para que tengan la forma deseada. 

1. Es necesario hacer el proceso inverso al one hot encoding para obtener el valor predicho
2. Los valores pudieron haberse salido de sus rangos normales, por ejemplo valores de mes de nacimiento de 15 (no existe el mes numero 15) o ingresos negativos. Por lo que se truncan estos valores

In [13]:
cols_SEX = names(modelo)[startsWith(names(modelo),'SEX')]
cols_POS_OCU = names(modelo)[startsWith(names(modelo),'POS_OCU')]
cols_CS_P13_1 = names(modelo)[startsWith(names(modelo),'CS_P13_1')]

Y_synt <- modelo %>% mutate (
        SEX = substr(cols_SEX[max.col(modelo[cols_SEX])],4,4),
        POS_OCU = substr(cols_POS_OCU[max.col(modelo[cols_POS_OCU])],8,8),
        CS_P13_1 = substr(cols_CS_P13_1[max.col(modelo[cols_CS_P13_1])],9,10)
    ) %>% mutate_at( 
        c(
            'SEX','POS_OCU', 'CS_P13_1'
        ),
        factor
    ) %>% mutate_at( 
        c(
            'EDA','NAC_DIA','NAC_MES','NAC_ANIO','INGOCUP'
        ),
        as.integer
    ) %>% mutate( 
        EDA = pmin(pmax(EDA,0),97),
        NAC_DIA = pmin(pmax(NAC_DIA,0),30),
        NAC_MES = pmin(pmax(NAC_MES,0),12),
        NAC_ANIO = pmin(pmax(NAC_ANIO,1921),2020),
        INGOCUP = pmax(INGOCUP,0),
    ) %>% select(
        all_of(names(Y))
    )

3. Finalmente, se integran los datos sinteticos $Y'$ con los datod no confidenciales $X$ para obtener la version sintetica (parcial) de los datos originales 

In [ ]:
sdem_interes_synt <- bind_cols(X,Y_synt)

head(sdem_interes_synt)

## Revisión de resultados

Podemos realizar una inspección visual de las distribuciones de cada una de las variables. Por lo que hacemos uso de la biblioteca [synthpop]( https://www.synthpop.org.uk/)

In [ ]:
library(synthpop)

options(repr.plot.width = 10, repr.plot.height =10)

Comparamos la distribucion original contra la sintetica

In [ ]:
compare(as.data.frame(sdem_interes_synt),as.data.frame(sdem_interes), vars= names(Y_synt), nrow = 3, ncol = 3)

Asi tambien podemos comparar relaciones entre variables. Por ejemplo, la relacion entre la edad y el año de nacimiento, la cual es conocida a priori.

In [ ]:
rbind(
    sdem_interes_synt %>% mutate(tipo = "synthetic"),
    sdem_interes %>% mutate(tipo = "observed")
)  %>% ggplot(aes(EDA, NAC_ANIO, shape=tipo, color=tipo)) + geom_point()